# Классификация изображений

В этом задании вам предстоит обучать несколько разных моделей для классификации изображений.

## Настройка google colab

Для корректной работы ноутбука и отправки заданий в тестирующую систему запустите следующие две ячейки (до начала работы).

In [ ]:
%%bash

shred -u setup_colab.py

wget -q https://raw.githubusercontent.com/hse-cs-ami/coursera-intro-dl/main/utils/colab_setup.py -O colab_setup.py

In [ ]:
import colab_setup


colab_setup.Week02CNN1().setup()

## Необходимые импорты

In [ ]:
from collections import defaultdict
from time import perf_counter
from typing import Type
from warnings import filterwarnings

import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.transforms as t

from IPython.display import clear_output
from testing import TestWeek02
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from tqdm import tqdm

filterwarnings('ignore')

sns.set(style='darkgrid')

## BaseModel

Базовый класс модели, от которого мы дальше будем наследовать все классы моделей.

In [ ]:
class BaseModel(nn.Module):

    def __init__(self, *args, **kwargs):
        super().__init__()

    def forward(self, x):
        raise NotImplementedError

## Wrapper

Класс-обёртка, который будет принимать нашу модель и обучать её на наборе данных CIFAR10. В процессе обучения будем строить графики и считать качество на обучающей и тестовой выборках.

In [ ]:
class Wrapper:

    def __init__(self, model_class: Type[BaseModel], train_transform: t.Compose = None, *model_args,
                 **model_kwargs) -> None:

        # задаём аугментации для обучающей и тестовой выборок
        if train_transform is None:
            train_transform: t.Compose = t.Compose(
                [
                    t.ToTensor(),
                    t.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ]
            )

        test_transform: t.Compose = t.Compose(
            [
                t.ToTensor(),
                t.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ]
        )

        # создаем датасеты
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=train_transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=test_transform)

        batch_size = 64

        # создаем даталоадеры
        self.train_loader: DataLoader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
        self.test_loader: DataLoader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

        self.device: torch.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        self.model: nn.Module = model_class(*model_args, **model_kwargs).to(self.device)

        # создаем функцию потерь и оптимизатор
        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = Adam(self.model.parameters(), lr=1e-3)

        self.history = defaultdict(list)

    def run(self, epochs: int = 10):
        total_time = 0

        for epoch in range(epochs):
            # строим графики
            self.plot_stats(epochs)

            # обучаемся
            start = perf_counter()
            self.train_epoch()
            total_time += perf_counter() - start

            # оцениваем качество
            self.evaluate_subset('train')
            self.evaluate_subset('test')

        # строим финальные графики и печатаем сколько заняло обучение
        self.plot_stats(epochs)
        print(f'Время на обучение: {total_time:.2f} секунд')

    def plot_stats(self, epochs) -> None:
        clear_output(wait=True)

        fig, (ax1, ax2) = plt.subplots(2, figsize=(10, 10), constrained_layout=True)

        ax1.plot(
            range(1, len(self.history['train_loss']) + 1),
            self.history['train_loss'],
            label='Обучающая', marker='^'
        )
        ax1.plot(
            range(1, len(self.history['test_loss']) + 1),
            self.history['test_loss'],
            label='Тестовая', marker='^'
        )

        ax1.set_xlim([0.5, epochs + 0.5])

        ax1.set_xlabel('Эпоха')
        ax1.set_ylabel('Ошибка')

        ax1.legend()

        ax2.plot(
            range(1, len(self.history['train_accuracy']) + 1),
            self.history['train_accuracy'],
            label='Обучающая', marker='^'
        )
        ax2.plot(
            range(1, len(self.history['test_accuracy']) + 1),
            self.history['test_accuracy'],
            label='Тестовая', marker='^'
        )

        ax2.set_xlim([0.5, epochs + 0.5])

        ax2.set_xlabel('Эпоха')
        ax2.set_ylabel('Доля правильных ответов')

        ax2.legend()

        plt.show()

    def train_epoch(self) -> None:
        self.model.train()

        for images, labels in tqdm(self.train_loader, desc='Обучение'):
            images, labels = images.to(self.device), labels.to(self.device)

            self.optimizer.zero_grad()

            outputs = self.model(images)

            loss = self.loss_function(outputs, labels)

            loss.backward()

            self.optimizer.step()

    @torch.no_grad()
    def evaluate_subset(self, subset: str = 'train') -> None:
        self.model.eval()

        loss = 0.0
        correct = 0

        loader: DataLoader = self.train_loader if subset == 'train' else self.test_loader

        for images, labels in tqdm(loader, desc=f'Оценка качества на {subset}'):
            images, labels = images.to(self.device), labels.to(self.device)

            outputs = self.model(images)

            loss += self.loss_function(outputs, labels).item()

            _, prediction = torch.max(outputs, 1)
            correct += prediction.eq(labels.data.view_as(prediction)).cpu().sum()

        self.history[f'{subset}_loss'].append(loss / len(loader))
        self.history[f'{subset}_accuracy'].append(100. * correct / len(loader.dataset))

## Задание 1

Допишите класс сети `MLP`, который представляет собой полносвязную нейронную сеть.
В вашей сети должно быть два скрытых слоя, каждый с 768 нейронов.

Используйте `nn.Linear` и `nn.Flatten`.

In [ ]:
class MLP(BaseModel):

    def __init__(self, *args, **kwargs):
        super().__init__()

        # На входе картинки 32 на 32 пикселя, 3 канала
        # На выходе 10 классов

        # Напишите составляющие нейронной сети по схеме из задания
        raise NotImplementedError

    def forward(self, x):
        # Напишите функцию прохода вперед
        raise NotImplementedError

In [ ]:
# инициализируем обертку
wrapper = Wrapper(model_class=MLP)

In [ ]:
# запускаем обучение
wrapper.run()

In [ ]:
# тестируем то, как вы написали MLP и насколько хорошо удалось обучить вашу нейронную сеть
TestWeek02.test01(wrapper)

## Задание 2

Допишите класс сети `CNN`, который представляет собой сверточную нейронную сеть.

Ваша сеть должна иметь следующую структуру:

 - Сверточный слой с 32 фильтрами, ядром 3 на 3 и паддингом 1.
 - Активация ReLU
 - MaxPool с ядром 2 на 2


 - Сверточный слой с 64 фильтрами, ядром 3 на 3 и паддингом 1.
 - Активация ReLU
 - MaxPool с ядром 2 на 2


 - Сверточный слой с 128 фильтрами, ядром 3 на 3 и паддингом 1.
 - Активация ReLU
 - MaxPool с ядром 2 на 2


 - Полносвязный слой с 10 выходными нейронами

In [ ]:
class CNN(BaseModel):

    def __init__(self, *args, **kwargs):
        super().__init__()

        # На входе картинки 32 на 32 пикселя, 3 канала
        # На выходе 10 классов

        # Напишите составляющие нейронной сети по схеме из задания
        raise NotImplementedError

    def forward(self, x):
        # Напишите функцию прохода вперед
        raise NotImplementedError

In [ ]:
# инициализируем обертку
wrapper = Wrapper(model_class=CNN)

In [ ]:
# запускаем обучение
wrapper.run()

In [ ]:
# тестируем то, как вы написали CNN и насколько хорошо удалось обучить вашу нейронную сеть
TestWeek02.test02(wrapper)

## Задание 3

Допишите класс сети `CNNpro`, который представляет собой сверточную нейронную сеть.

Ваша сеть должна иметь следующую структуру:

 - Сверточный слой с 32 фильтрами, ядром 3 на 3 и паддингом 1.
 - Активация ReLU
 - Батч-нормализация
 - MaxPool с ядром 2 на 2
 - Дропаут с вероятностью удаления 0.2


 - Сверточный слой с 64 фильтрами, ядром 3 на 3 и паддингом 1.
 - Активация ReLU
 - Батч-нормализация
 - MaxPool с ядром 2 на 2
 - Дропаут с вероятностью удаления 0.2


 - Сверточный слой с 128 фильтрами, ядром 3 на 3 и паддингом 1.
 - Активация ReLU
 - Батч-нормализация
 - MaxPool с ядром 2 на 2
 - Дропаут с вероятностью удаления 0.2


 - Полносвязный слой с 10 выходными нейронами

In [ ]:
class CNNpro(BaseModel):

    def __init__(self, *args, **kwargs):
        super().__init__()

        # На входе картинки 32 на 32 пикселя, 3 канала
        # На выходе 10 классов

        # Напишите составляющие нейронной сети по схеме из задания
        raise NotImplementedError

    def forward(self, x):
        # Напишите функцию прохода вперед
        raise NotImplementedError

In [ ]:
# инициализируем обертку
wrapper = Wrapper(model_class=CNNpro)

In [ ]:
# запускаем обучение
wrapper.run()

In [ ]:
# тестируем то, как вы написали CNNpro и насколько хорошо удалось обучить вашу нейронную сеть
TestWeek02.test03(wrapper)

## Задание 4

Добавим аугментаций!

Попробуйте использовать какие-нибудь аугментации из списка:

 - [RandomHorizontalFlip](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomHorizontalFlip)
 - [RandomAffine](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomAffine)
 - [ColorJitter](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ColorJitter)
 - [RandomGrayscale](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomGrayscale)
 - [RandomCrop](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomCrop)

Или используйте любую другую аугментацию по вашему вкусу.

In [ ]:
custom_transform = t.Compose(
    [
        # добавьте сюда аугментации из списка выше
        t.ToTensor(),
        t.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [ ]:
# инициализируем обертку
wrapper = Wrapper(model_class=CNNpro, transform=custom_transform)

In [ ]:
# запускаем обучение
wrapper.run()

In [ ]:
# тестируем ваши аугментации
TestWeek02.test04(wrapper)